
<img src="https://raw.githubusercontent.com/rhatiro/Curso_EBAC-Profissao_Cientista_de_Dados/main/ebac-course-utils/media/logo/newebac_logo_black_half.png" alt="ebac-logo">

---

# **Profissão: Cientista de Dados**
### **Módulo 13** | Regressão II | Exercício 2

Aluno [Roberto Hatiro Nishiyama](https://www.linkedin.com/in/rhatiro/)<br>
Data: 31 de março de 2023.

---

# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

import patsy
import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score

In [2]:
# df = pd.read_csv('previsao_de_renda.csv')
filepath = 'https://raw.githubusercontent.com/rhatiro/Curso_EBAC-Profissao_Cientista_de_Dados/main/M%C3%B3dulo%2013%20-%20Regress%C3%A3o%20II/database/previsao_de_renda.csv'

df = pd.read_csv(filepath_or_buffer=filepath)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

In [4]:
df = df.iloc[:,3:]
df

,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
0,F,False,True,0,Empresário,Secundário,Solteiro,Casa,26,6.602740,1.0,8060.34
1,M,True,True,0,Assalariado,Superior completo,Casado,Casa,28,7.183562,2.0,1852.15
2,F,True,True,0,Empresário,Superior completo,Casado,Casa,35,0.838356,2.0,2253.89
3,F,False,True,1,Servidor público,Superior completo,Casado,Casa,30,4.846575,3.0,6600.77
4,M,True,False,0,Assalariado,Secundário,Solteiro,Governamental,33,4.293151,1.0,6475.97
...,...,...,...,...,...,...,...,...,...,...,...,...
14995,F,False,True,0,Empresário,Secundário,Solteiro,Casa,48,13.887671,1.0,7990.58
14996,F,False,True,0,Pensionista,Superior completo,Solteiro,Casa,57,NaN,1.0,10093.45
14997,F,True,True,0,Assalariado,Superior completo,Casado,Casa,45,7.832877,2.0,604.82
14998,M,True,False,0,Empresário,Superior completo,Casado,Casa,36,4.298630,2.0,3352.27


1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na base de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

#### 1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).

In [5]:
df_train, df_test = train_test_split(df, test_size=0.25, random_state=42)

In [6]:
modelo = '''renda ~ sexo 
                    + posse_de_veiculo 
                    + posse_de_imovel 
                    + qtd_filhos 
                    + tipo_renda 
                    + educacao 
                    + estado_civil 
                    + tipo_residencia 
                    + idade 
                    + tempo_emprego 
                    + qt_pessoas_residencia
                    '''

y_train, X_train = patsy.dmatrices(formula_like=modelo, data=df_train)
y_test, X_test = patsy.dmatrices(formula_like=modelo, data=df_test)

#### 2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?

In [7]:
alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]
r2_y_pred = []

for alpha in alphas:
    md = sm.OLS(y_train, X_train)
    reg = md.fit_regularized(method='elastic_net', 
                             refit=True, 
                             L1_wt=0,  # ridge fit
                             alpha=alpha)
    y_pred = reg.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    
    r2_y_pred.append(r2)
    print(f'Alpha {alpha}: \nR-squared = {r2}\n')
    
pd.DataFrame({'alpha':alphas, 
              '𝑅2':r2_y_pred
             }).sort_values(by='𝑅2', ascending=False)

Alpha 0: 
R-squared = 0.2624340344281858

Alpha 0.001: 
R-squared = 0.262261949523507

Alpha 0.005: 
R-squared = 0.26196637760082875

Alpha 0.01: 
R-squared = 0.2615684758582866

Alpha 0.05: 
R-squared = 0.25638277607326576

Alpha 0.1: 
R-squared = 0.24863408637051077



,alpha,𝑅2
0,0.000,0.262434
1,0.001,0.262262
2,0.005,0.261966
3,0.010,0.261568
4,0.050,0.256383
5,0.100,0.248634


Entre os modelos de regressão com regularização *ridge* testados, o que obteve o melhor coeficiente de determinação foi aquele que utilizou um valor de alpha igual a 0.000, resultando em um 𝑅2 de 0.262434.

#### 3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?

In [8]:
alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]
r2_y_pred = []

for alpha in alphas:
    md = sm.OLS(y_train, X_train)
    reg = md.fit_regularized(method='elastic_net', 
                             refit=True, 
                             L1_wt=1,  # lasso fit
                             alpha=alpha)
    y_pred = reg.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    
    r2_y_pred.append(r2)
    print(f'Alpha {alpha}: \nR-squared = {r2}\n')
    
pd.DataFrame({'alpha':alphas, 
              '𝑅2':r2_y_pred
             }).sort_values(by='𝑅2', ascending=False)

Alpha 0: 
R-squared = 0.2624340344281857

Alpha 0.001: 
R-squared = 0.2624340344281857

Alpha 0.005: 
R-squared = 0.2624340344281857

Alpha 0.01: 
R-squared = 0.2624340344281857

Alpha 0.05: 
R-squared = 0.26250010213196706

Alpha 0.1: 
R-squared = 0.2624340344281857



,alpha,𝑅2
4,0.050,0.262500
0,0.000,0.262434
1,0.001,0.262434
2,0.005,0.262434
3,0.010,0.262434
5,0.100,0.262434


Dentre os modelos de regressão com regularização *LASSO* testados, aquele que utilizou um valor de alpha igual a 0.050 obteve o melhor coeficiente de determinação, com um 𝑅2 de 0.2625.

#### 4. Rode um modelo *stepwise*. Avalie o $R^2$ na base de testes. Qual o melhor resultado?

In [9]:
df_dummies = pd.get_dummies(data=df.dropna(), drop_first=True).astype(int)

X = df_dummies.drop(columns='renda')
y = df_dummies['renda']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [10]:
def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.05, 
                       threshold_out = 0.05, 
                       verbose=True):
    """ Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    included = list(initial_list)
    while True:
        changed=False
        
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded, dtype=np.dtype('float64'))
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.index[new_pval.argmin()]
            included.append(best_feature)
            changed=True
            if verbose:
                 print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        print("#############")
        print(included)
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.argmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

variaveis = stepwise_selection(X_test, y_test)

print('\n\nresulting features:')
print(variaveis)

Add  tempo_emprego                  with p-value 1.15416e-120
#############
['tempo_emprego']
Add  sexo_M                         with p-value 1.97372e-116
#############
['tempo_emprego', 'sexo_M']
Add  educacao_Superior completo     with p-value 0.00030172
#############
['tempo_emprego', 'sexo_M', 'educacao_Superior completo']
Add  idade                          with p-value 0.000294673
#############
['tempo_emprego', 'sexo_M', 'educacao_Superior completo', 'idade']
Add  posse_de_veiculo               with p-value 0.00268263
#############
['tempo_emprego', 'sexo_M', 'educacao_Superior completo', 'idade', 'posse_de_veiculo']
Add  tipo_renda_Empresário          with p-value 0.0127891
#############
['tempo_emprego', 'sexo_M', 'educacao_Superior completo', 'idade', 'posse_de_veiculo', 'tipo_renda_Empresário']
Add  qt_pessoas_residencia          with p-value 0.0167073
#############
['tempo_emprego', 'sexo_M', 'educacao_Superior completo', 'idade', 'posse_de_veiculo', 'tipo_renda_Empresário

In [11]:
X = df_dummies[variaveis]
y = df_dummies['renda']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

y_pred = sm.OLS(y_train, X_train).fit().predict(X_test)
print(f'R-squared: {r2_score(y_test, y_pred)}')

R-squared: 0.29010227899726104


In [12]:
alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]
r2_y_pred = []

for alpha in alphas:
    md = sm.OLS(y_train, X_train)
    reg = md.fit_regularized(method='elastic_net', 
                             refit=True, 
                             L1_wt=0,  # ridge fit
                             alpha=alpha)
    y_pred = reg.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    
    r2_y_pred.append(r2)
    print(f'Alpha {alpha}: \nR-squared = {r2}\n')
    
pd.DataFrame({'alpha':alphas, 
              '𝑅2':r2_y_pred
             }).sort_values(by='𝑅2', ascending=False)

Alpha 0: 
R-squared = 0.29010227899726104

Alpha 0.001: 
R-squared = 0.2902825082446059

Alpha 0.005: 
R-squared = 0.29085279202395087

Alpha 0.01: 
R-squared = 0.2913441992484578

Alpha 0.05: 
R-squared = 0.29077212194567714

Alpha 0.1: 
R-squared = 0.28521133849996116



,alpha,𝑅2
3,0.010,0.291344
2,0.005,0.290853
4,0.050,0.290772
1,0.001,0.290283
0,0.000,0.290102
5,0.100,0.285211


In [13]:
alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]
r2_y_pred = []

for alpha in alphas:
    md = sm.OLS(y_train, X_train)
    reg = md.fit_regularized(method='elastic_net', 
                             refit=True, 
                             L1_wt=1,  # lasso fit
                             alpha=alpha)
    y_pred = reg.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    
    r2_y_pred.append(r2)
    print(f'Alpha {alpha}: \nR-squared = {r2}\n')
    
pd.DataFrame({'alpha':alphas, 
              '𝑅2':r2_y_pred
             }).sort_values(by='𝑅2', ascending=False)

Alpha 0: 
R-squared = 0.29010227899726104

Alpha 0.001: 
R-squared = 0.29010227899726104

Alpha 0.005: 
R-squared = 0.29010227899726104

Alpha 0.01: 
R-squared = 0.29010227899726104

Alpha 0.05: 
R-squared = 0.29010227899726104

Alpha 0.1: 
R-squared = 0.29010227899726104



,alpha,𝑅2
0,0.000,0.290102
1,0.001,0.290102
2,0.005,0.290102
3,0.010,0.290102
4,0.050,0.290102
5,0.100,0.290102


Na minha opinião, o melhor resultado dos modelos stepwise foi o que utilizou a regressão *LASSO*, pois obteve desempenho semelhante em todos os valores de alpha e ainda assim alcançou um coeficiente de determinação comparável ao modelo que utilizou a regularização ridge com o maior valor desse coeficiente. Dessa forma, esse modelo pode ser considerado mais simples e, ao mesmo tempo, apresentar o melhor resultado entre as opções desenvolvidas.

#### 5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?

A melhor opção dentre os modelos mencionados é o modelo de regressão *LASSO* com seleção de variáveis do tipo stepwise. Mesmo utilizando um número menor de variáveis em comparação aos outros modelos, este modelo foi capaz de obter o maior coeficiente de determinação, apresentando pouca variação nos resultados entre os parâmetros.

#### 6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.

In [14]:
df_matrix = df_dummies.loc[:, variaveis + ['renda']]
df_matrix.rename(columns={'educacao_Superior completo': 'educacao_Superior_completo'}, 
                 inplace=True)

df_train, df_test = train_test_split(df_matrix, test_size=0.25, random_state=42)

modelo = smf.ols(formula='''
                         np.log(renda) ~ tempo_emprego 
                                         + sexo_M 
                                         + educacao_Superior_completo
                                         + idade 
                                         - posse_de_veiculo 
                                         + tipo_renda_Empresário 
                                         + qt_pessoas_residencia
                                         - tipo_residencia_Estúdio
                                         ''', data=df_train)

reg = modelo.fit_regularized(method='elastic_net', 
                             refit=True, 
                             L1_wt=1,
                             alpha=0)

print(reg.summary())

y_pred = reg.predict(df_test)
print('\nCoeficiente de determinação (𝑅2) na base de testes:', 
      r2_score(df_test['renda'], 
               np.exp(y_pred)
              )
     )

                            OLS Regression Results                            
Dep. Variable:          np.log(renda)   R-squared:                       0.348
Model:                            OLS   Adj. R-squared:                  0.347
Method:                 Least Squares   F-statistic:                     709.8
Date:                Fri, 31 Mar 2023   Prob (F-statistic):               0.00
Time:                        00:27:09   Log-Likelihood:                -10235.
No. Observations:                9320   AIC:                         2.049e+04
Df Residuals:                    9313   BIC:                         2.054e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

#### 7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

In [15]:
X = df_dummies[variaveis]
y = df_dummies['renda']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

reg_tree = DecisionTreeRegressor(random_state=42, max_depth=5, min_samples_leaf=13)
reg_tree.fit(X_train, y_train)

DecisionTreeRegressor(max_depth=5, min_samples_leaf=13, random_state=42)

In [16]:
print('Coeficiente de determinação (𝑅2) na base de testes:', 
      reg_tree.score(X=X_test, y=y_test))

Coeficiente de determinação (𝑅2) na base de testes: 0.3890548749887145


---